In [1]:
from tensorflow import keras


model1 = keras.models.load_model('../models/pre_model_1_1_10_epochs.h5')
model2 = keras.models.load_model('../models/pre_model_2_1.h5')
model5 = keras.models.load_model('../models/pre_model_5_1_10_epochs.h5')
model6 = keras.models.load_model('../models/pre_model_6_1_20_2_epochs.h5')
model7 = keras.models.load_model('../models/pre_model_7_1_10_epochs.h5')


In [2]:
all_models = [model1,model2, model5,model6,model7]
rule_names = rules = [
            "Kein Scooter",
            "Scooter unvollständig/zu wenig Umgebung",
            "Scooter auf Grünfläche",
            "Scooter blockiert Fahrradständer",
            "Scooter steht an ÖPNV Haltestelle / Parkverbot",
        ]

In [3]:
from PIL import Image
import numpy as np
# Function to load and resize an image
def load_image(img_path):
    with Image.open(img_path) as img:
        img = img.resize((512, 512))
        img = img.convert('RGB')  # Ensure image is in RGB format
        return np.array(img)


In [5]:
import matplotlib.pyplot as plt

def show_img(img_array):
    img_array.astype('uint8')
    plt.imshow(img_array)
    plt.axis('off')  # Hide the axes
    plt.show()

In [6]:
import os
import random

folder_path = "../Yoio_Park_Proof"
all_files = os.listdir(folder_path)
random_image_file = random.choice(all_files)
img_path = os.path.join(folder_path, random_image_file)
img = load_image(img_path)
img_array = np.expand_dims(img, axis=0)
print(img.shape)

(512, 512, 3)


In [7]:
import os
import random

folder_path = "../Yoio_Park_Proof"
all_files = os.listdir(folder_path)
random_image_file = random.choice(all_files)
img_path = os.path.join(folder_path, random_image_file)

def predict_image(models,img_path):


    img = load_image(img_path)

    for model in models:
        img_array = np.expand_dims(img, axis=0)
        predictions = model.predict(img_array)
        
        #Model 4 predicted Falsch herum i guess
        if models.index(model) !=5 :
            #if predictions[0][0] > predictions[0][1]: Wäre eine Option
            if predictions[0][0]>0.95:
                print("Verstoß")
                print(rules[models.index(model)])
                break
            else:
                print("Richtig")
        else: 
            if predictions[0][1]>0.95:
                print("Verstoß")
                print(rules[models.index(model)])
                break
            else:
                print("Richtig")


    show_img(img)
    

In [4]:
import pandas as pd
balanced_labels = pd.read_csv('../all_models_combine/chosen_labels_not_all_rules.csv', sep=';',index_col=False)
balanced_labels.sample(5)

,Image_name,Rule1,Rule2,Rule5,Rule6,Rule7
193,1612112016ef696dbdc6032fbb7fbe9dec01b9d1bb4324...,1,0,0,0,0
219,1601958665af79f47ed2ad422a53fff0540e6349e13fe8...,1,0,0,0,0
161,15861873084a625290bddf2ca11d70157e0613cc8c13f5...,0,0,0,0,0
143,1604574672fd87ad09106bf279d80c9350d3b2c99d8772...,0,0,0,0,0
522,15839228301d85f8699e86e3a6b84d80ba0815fe4e8846...,0,0,0,1,0


In [6]:

columns=["Image_name","Rule1","Rule2","Rule5","Rule6","Rule7"]
predictions_df = pd.DataFrame(columns=columns)

i=0
while i <= min(1000, len(balanced_labels)):
    folder_path = "../Yoio_Park_Proof/"
    img_name=str(balanced_labels.sample(1).iloc[0,0])


    img = load_image(folder_path+img_name)

    predictions_array =[img_name]

    for model in all_models:
        img_array = np.expand_dims(img, axis=0)
        predictions = model.predict(img_array)
        #Model 4 predicted Falsch herum i guess
        if all_models.index(model) !=3 :
            #if predictions[0][0] > predictions[0][1]: Wäre eine Option
            if predictions[0][0]>0.95:
                int_pred=1
            else:
                int_pred=0
        else: 
            if predictions[0][1]>0.95:
                int_pred=1
            else:
                int_pred=0
        predictions_array.append(int_pred)

    predictions_df.loc[len(predictions_df)] = predictions_array
    #new_row = pd.DataFrame(predictions_array, columns=columns)
    #predictions_df = pd.concat([predictions_array,new_row],ignore_index=True)

    i += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━

In [7]:
print(predictions_df.sample(10))

                                            Image_name  Rule1  Rule2  Rule5  \
195  1599920333a4a54272a145ae896935ee4a78066956f277...      0      1      0   
838  16150368328ad96930bbac77f8c6aeef3c50326fa7268a...      0      1      0   
231  160795908029d2e6ea0eca16ef4a0e8fb82b77b9674c54...      0      1      0   
694  16106504639d1e06dde7b1356be9098b837cecec33bd65...      0      0      0   
340  1610813662b11577a3f0af7d8d984e047bbb4dae92bf6d...      0      1      0   
658  160925040226de200781276a21642a324571953590df34...      0      1      0   
798  16074484198f98e4151944a3ca123c3ac10e0557146c81...      0      1      0   
218  1601359630ecbde134b03fc0f01cab5c2446787a1e8c24...      0      1      0   
158  161459946339104eb2d2021fdd5261c7c4512a03b609f1...      0      1      0   
297  1609362905f3884c87b2bb3e69f30c87e2c0c704d5dfd8...      0      1      1   

     Rule6  Rule7  
195      0      0  
838      0      0  
231      0      0  
694      0      0  
340      0      0  
658      0

In [8]:
predictions_df.to_csv('../all_models_combine/predicted_labels_not_all_rules.csv', sep=';', index=False)